In [16]:
import requests
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
import pandas as pd 
from sklearn.decomposition import PCA

api_key = 'f76546bce033451fb83c2e807a3763ff'
url = f'https://newsapi.org/v2/everything'
params = {
    'q': 'cine',
    'language': 'es', 
    'apiKey': api_key  
}
response = requests.get(url, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    articles = data.get('articles', [])
    
    # Contar el número de noticias
    num_articles = len(articles)
    print(f"Se encontraron {num_articles} noticias sobre {params['q']}.")

    for article in articles:
        print(f"- {article['title']}")
else:
    print("Error al obtener las noticias:", response.status_code)


Se encontraron 100 noticias sobre cine.
- Esta película es una de las mejores de 2025 y está destrozando récords sin parar
- El CEO de Netflix, sin tapujos: dice que ir al cine es obsoleto y que su plataforma está salvando a Hollywood
- Movistar Plus+ da la sorpresa y lanza tres canales nuevos de cine y series
- Las películas más extrañas estrenadas en 2025
- 7 películas blasfemas para Semana Santa
- 5 sagas de terror que puedes ver completas en streaming
- ‘Amateur’, crítica: Un ‘Misión Imposible’ más cerebral pero menos divertido
- El director de ‘Una película de Minecraft’ se pronuncia por la polémica locura del «Chicken Jockey»
- ‘Until Dawn’, crítica: La adaptación del juego se queda a medias
- Kojima dice que la película de ‘Death Stranding’ será «extraña», para sorpresa de nadie
- ‘El contable 2’, crítica: Una secuela más madura y divertida
- La extraña y controvertida historia de real de Ed y Lorraine Warren, en la que se basa ‘The Conjuring’
- ‘Warfare’, crítica: Alex Garland 

In [17]:
df=pd.DataFrame(articles)
df=df['content']
df.head()

0    Este 2025 ya cuenta con una de esas películas ...
1    Ted Sarandos, codirector ejecutivo de Netflix,...
2    Movistar continúa haciendo cambios en Movistar...
3    Con toda seguridad, 2025 será recordado como e...
4    No, no era una de esas películas de Semana San...
Name: content, dtype: object

In [18]:
#preprocesar datos
def preprocesador(text):
    text = re.sub(r'[\W]+', ' ', text.lower())
    text = re.sub(r'\d','', text)
    text = text.strip()
    return text

df=df.apply(preprocesador)
df.shape
#vectorizar en bigramas con bolsa de palabras, usando "bow_vectorizer = CountVectorizer(...)"
Stopwords_es=nltk.corpus.stopwords.words('spanish')

bow_vectorizer = CountVectorizer(ngram_range=(1,2),max_df=0.90 ,min_df=2,stop_words=Stopwords_es)

# ngram_range=(1,2) para unigramas y bigramas # cantidad minima(primera entrada) de ngramas y maxima(segunda entrada)
# max_df=0.95 para eliminar palabras que aparecen en más del 95% de los documentos
# min_df=2 para eliminar palabras que aparecen en menos de 2 documentos

bow=bow_vectorizer.fit_transform(df)
bow.shape
lexico = bow_vectorizer.get_feature_names()


In [19]:
pca = PCA(n_components=100)
pca = pca.fit(bow.toarray())

pca_topic_vectors = pca.transform(bow.toarray()) 
pca_topic_vectors.shape 

n_top_words = 7
feature_names = bow_vectorizer.get_feature_names() #lexico

In [20]:

top_words_by_topic = []
for component_idx, component in enumerate(pca.components_):
    top_features_idx = component.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_idx]
    top_words_by_topic.append([f"Tópico {component_idx + 1}"] + top_features)
    print(f"Tópico {component_idx + 1}: {', '.join(top_features)}")


Tópico 1: cine, huinca cine, huinca, productora, flexible, publicitaria, productora huinca
Tópico 2: película, película minecraft, minecraft, cines, videojuego, segunda, nueva
Tópico 3: años, wars, star, star wars, director, wars celebration, celebration
Tópico 4: películas, haciendo, movistar, cines, estrena, plataforma, movistar continúa
Tópico 5: cine, estrena, nueva, nueva película, cines, cinematográfico marvel, cines thunderbolts
Tópico 6: años, llega, director, affleck, ben, contable, connor
Tópico 7: ciencia, ficción, ciencia ficción, historia, creado actualizado, creado, actualizado
Tópico 8: quieres, si, si quieres, haciendo, tele, movistar, samsung
Tópico 9: ciencia, ficción, ciencia ficción, películas, años, película, quieres
Tópico 10: películas, quieres, si, si quieres, minecraft, tele, hollywood
Tópico 11: final, ser, cines, saga, estreno, cruise, tom
Tópico 12: estrena cines, después, alex, alex garland, cines warfare, garland, warfare
Tópico 13: ser, semana, taquilla, 

In [21]:
df_top_words = pd.DataFrame(top_words_by_topic, columns=["Tópico"] + [f"Palabra {i+1}" for i in range(n_top_words)])
df_top_words.to_csv('top_words_by_topic.csv', index=False)

print("Las palabras más comunes por tópico se han guardado en 'top_words_by_topic.csv'.")

Las palabras más comunes por tópico se han guardado en 'top_words_by_topic.csv'.
